In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

A custom environment must define these core methods:

* __init__: Initialize the environment.
* reset: Reset the environment to its initial state and return the initial observation.
* step: Take an action and return (observation, reward, done, truncated, info).
* render (optional): Visualize the environment (if needed).
* close (optional): Clean up resources.

### This is environment that enable the drone to follow the balloon. In this environment, the ball didn't respawn randomly

In [66]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class DroneChaseEnv(gym.Env):
    metadata = {'render_modes': ['human']}
    
    def __init__(self, render_mode=None):
        super(DroneChaseEnv, self).__init__()
        
        # Canvas size
        self.width = 600
        self.height = 400
        
        # Score
        self.score = 0  # Tracks how many times the drone hits the balloon

        # Drone parameters
        self.drone_pos = np.array([self.width / 2, self.height / 2], dtype=np.float32)
        self.drone_vel = np.array([0.0, 0.0], dtype=np.float32)
        self.max_speed = 5.0

        # Balloon parameters
        self.balloon_pos = np.array([np.random.uniform(0, self.width), 
                                     np.random.uniform(0, self.height)], dtype=np.float32)
        self.balloon_vel = np.array([np.random.uniform(-1, 1), 
                                     np.random.uniform(-1, 1)], dtype=np.float32)

        # Action space: Continuous thrust adjustments for left and right propellers
        # Thrust values between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)

        # Observation space: Positions and velocities
        # Drone position (x, y), Drone velocity (vx, vy), Balloon position (x, y)
        low_obs = np.array([0, 0, -self.max_speed, -self.max_speed, 0, 0], dtype=np.float32)
        high_obs = np.array([self.width, self.height, self.max_speed, self.max_speed, self.width, self.height], dtype=np.float32)
        self.observation_space = spaces.Box(low=low_obs, high=high_obs, dtype=np.float32)

        # Rendering
        self.render_mode = render_mode
        if self.render_mode == 'human':
            import pygame
            pygame.init()
            self.screen = pygame.display.set_mode((self.width, self.height))
            self.clock = pygame.time.Clock()

        # Episode parameters
        self.max_steps = 200
        self.current_step = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        # Reset drone position and velocity
        self.drone_pos = np.array([self.width / 2, self.height / 2], dtype=np.float32)
        self.drone_vel = np.array([0.0, 0.0], dtype=np.float32)

        # Reset balloon position and velocity
        self.balloon_pos = np.array([np.random.uniform(0, self.width), 
                                     np.random.uniform(0, self.height)], dtype=np.float32)
        self.balloon_vel = np.array([np.random.uniform(-1, 1), 
                                     np.random.uniform(-1, 1)], dtype=np.float32)

        self.current_step = 0

        observation = self._get_obs()
        info = {}

        return observation, info

    def step(self, action):
        # Update drone velocity based on action (thrust adjustments)
        left_thrust, right_thrust = action
        thrust = np.array([right_thrust - left_thrust, (left_thrust + right_thrust) / 2], dtype=np.float32)
        
        # Update drone velocity and position
        self.drone_vel += thrust
        
        # Limit speed
        speed = np.linalg.norm(self.drone_vel)
        if speed > self.max_speed:
            self.drone_vel = (self.drone_vel / speed) * self.max_speed
        self.drone_pos += self.drone_vel

        # Update balloon position
        self.balloon_pos += self.balloon_vel
        
        # Keep balloon within bounds
        self.balloon_pos = np.clip(self.balloon_pos, [0, 0], [self.width, self.height])

        # Keep drone within bounds
        self.drone_pos = np.clip(self.drone_pos, [0, 0], [self.width, self.height])

        # Compute distance to balloon
        distance = np.linalg.norm(self.drone_pos - self.balloon_pos)

        # Reward: Negative distance to balloon
        reward = -distance * 0.01  # Scale down the reward

        # Check if the drone catches the balloon (within a certain radius)
        done = False
        catch_radius = 10.0
        if distance < catch_radius:
            reward += 10.0  # Bonus for catching the balloon
            self.score += 1  # Increment the score
            done = True

        # Penalty for each time step
        reward -= 0.1

        # Increment step count
        self.current_step += 1
        if self.current_step >= self.max_steps:
            done = True

        observation = self._get_obs()
        info = {}

        if self.render_mode == 'human':
            self.render()

        return observation, reward, done, False, info

    def render(self):
        if self.render_mode != 'human':
            return

        import pygame
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        self.screen.fill((255, 255, 255))  # White background

        # Draw balloon
        pygame.draw.circle(self.screen, (255, 0, 0), self.balloon_pos.astype(int), 10)  # Red circle

        # Draw drone
        pygame.draw.rect(self.screen, (0, 0, 255), (*self.drone_pos - 10, 20, 20))  # Blue square

        # Add labels
        font = pygame.font.Font(None, 24)  # Default font, size 24
        balloon_label = font.render("Balloon", True, (0, 0, 0))  # Black text for balloon
        drone_label = font.render("Drone", True, (0, 0, 0))  # Black text for drone

        # Position labels near objects
        self.screen.blit(balloon_label, (self.balloon_pos[0] - 20, self.balloon_pos[1] - 20))
        self.screen.blit(drone_label, (self.drone_pos[0] - 20, self.drone_pos[1] - 20))
        
        # Display timer
        time_remaining = max(0, 30 - self.current_step / 30)  # 30 FPS assumed
        timer_label = font.render(f"Time Remaining: {time_remaining:.1f}s", True, (0, 0, 0))
        self.screen.blit(timer_label, (10, 10))
        
        # Display the score
        score_label = font.render(f"Score: {self.score}", True, (0, 0, 0))
        self.screen.blit(score_label, (10, 40))  # Display below the timer
        
        pygame.display.flip()
        self.clock.tick(30)

    def close(self):
        if self.render_mode == 'human':
            import pygame
            pygame.quit()

    def _get_obs(self):
        return np.concatenate((self.drone_pos, self.drone_vel, self.balloon_pos)).astype(np.float32)


### This is environment that enable the drone to follow the balloon. In this environment, the ball will respawn randomly

In [2]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class DroneChaseEnv(gym.Env):
    metadata = {'render_modes': ['human']}
    
    def __init__(self, render_mode=None):
        super(DroneChaseEnv, self).__init__()
        
        # Canvas size
        self.width = 600
        self.height = 400
        
        # Score
        self.score = 0  # Tracks how many times the drone hits the balloon

        # Drone parameters
        self.drone_pos = np.array([self.width / 2, self.height / 2], dtype=np.float32)
        self.drone_vel = np.array([0.0, 0.0], dtype=np.float32)
        self.max_speed = 5.0

        # Balloon parameters
        self.balloon_pos = np.array([np.random.uniform(0, self.width), 
                                     np.random.uniform(0, self.height)], dtype=np.float32)
        self.balloon_vel = np.array([np.random.uniform(-1, 1), 
                                     np.random.uniform(-1, 1)], dtype=np.float32)

        # Action space: Continuous thrust adjustments for left and right propellers
        # Thrust values between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)

        # Observation space: Positions and velocities
        # Drone position (x, y), Drone velocity (vx, vy), Balloon position (x, y)
        low_obs = np.array([0, 0, -self.max_speed, -self.max_speed, 0, 0], dtype=np.float32)
        high_obs = np.array([self.width, self.height, self.max_speed, self.max_speed, self.width, self.height], dtype=np.float32)
        self.observation_space = spaces.Box(low=low_obs, high=high_obs, dtype=np.float32)

        # Rendering
        self.render_mode = render_mode
        if self.render_mode == 'human':
            import pygame
            pygame.init()
            self.screen = pygame.display.set_mode((self.width, self.height))
            self.clock = pygame.time.Clock()

        # Episode parameters
        self.max_steps = 200
        self.current_step = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        # Reset drone position and velocity
        self.drone_pos = np.array([self.width / 2, self.height / 2], dtype=np.float32)
        self.drone_vel = np.array([0.0, 0.0], dtype=np.float32)

        # Reset balloon position and velocity
        self.balloon_pos = np.array([np.random.uniform(0, self.width), 
                                     np.random.uniform(0, self.height)], dtype=np.float32)
        self.balloon_vel = np.array([np.random.uniform(-1, 1), 
                                     np.random.uniform(-1, 1)], dtype=np.float32)

        self.current_step = 0

        observation = self._get_obs()
        info = {}

        return observation, info

    def step(self, action):
        """
        Executes one time step in the environment.

        Args:
            action (array): Array of two thrust values [left_thrust, right_thrust], 
                            where each value is between -1 and 1.

        Returns:
            observation (array): Updated state containing [drone_position, drone_velocity, balloon_position].
            reward (float): The reward for the current step.
            done (bool): Whether the episode has ended (always False for continuous simulation).
            truncated (bool): Whether the episode was truncated (always False here).
            info (dict): Additional debugging information (empty here).
        """

        # 1. Calculate the drone's thrust based on the action
        # The left and right thrust values determine the horizontal and vertical motion of the drone.
        left_thrust, right_thrust = action
        thrust = np.array([right_thrust - left_thrust,  # Horizontal movement (right thrust - left thrust)
                        (left_thrust + right_thrust) / 2],  # Vertical movement (average thrust)
                        dtype=np.float32)

        # 2. Update the drone's velocity based on the thrust
        self.drone_vel += thrust

        # 3. Enforce the maximum speed limit
        # If the drone's speed exceeds the allowed max_speed, scale it down to max_speed.
        speed = np.linalg.norm(self.drone_vel)  # Calculate the magnitude of the velocity vector
        if speed > self.max_speed:
            self.drone_vel = (self.drone_vel / speed) * self.max_speed  # Scale the velocity vector

        # 4. Update the drone's position based on the velocity
        self.drone_pos += self.drone_vel

        # 5. Ensure the drone stays within the canvas bounds
        # If the position goes outside the defined environment, clip it to the boundaries.
        self.drone_pos = np.clip(self.drone_pos, [0, 0], [self.width, self.height])

        # 6. Calculate the distance between the drone and the balloon
        # Use the Euclidean distance formula to determine how far the drone is from the balloon.
        distance = np.linalg.norm(self.drone_pos - self.balloon_pos)

        # 7. Check if the drone catches the balloon (within the catch radius)
        catch_radius = 10.0  # Defines the radius within which the drone "catches" the balloon
        if distance < catch_radius:
            # If the drone catches the balloon:
            # - Increment the score
            self.score += 1
            # - Respawn the balloon at a random location
            self.balloon_pos = np.random.uniform([0, 0], [self.width, self.height])
            # - Recalculate the distance to the new balloon position
            distance = np.linalg.norm(self.drone_pos - self.balloon_pos)
            # - Give a large reward for catching the balloon
            reward = 10.0
        else:
            # If the balloon is not caught, penalize the drone based on the distance
            # Closer to the balloon = less penalty; farther away = more penalty
            reward = -distance * 0.01

        # 8. Randomly move the balloon
        # The balloon moves in small, random steps within the environment bounds.
        self.balloon_pos += np.random.uniform(-2, 2, size=(2,))
        self.balloon_pos = np.clip(self.balloon_pos, [0, 0], [self.width, self.height])  # Keep balloon in bounds

        # 9. Penalize the drone for each time step to encourage efficiency
        reward -= 0.1

        # 10. Increment the step count
        # Track how many steps have been taken in the episode.
        self.current_step += 1

        # 11. Create the observation array
        # The observation includes the drone's position, velocity, and the balloon's position.
        observation = self._get_obs()

        # 12. Return the updated state, reward, and additional info
        # 'done' and 'truncated' are False here because this is a continuous simulation.
        info = {}
        if self.render_mode == 'human':
            self.render()
        return observation, reward, False, False, info

    def render(self):
        if self.render_mode != 'human':
            return

        import pygame
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        self.screen.fill((255, 255, 255))  # White background

        # Draw balloon
        pygame.draw.circle(self.screen, (255, 0, 0), self.balloon_pos.astype(int), 10)  # Red circle

        # Draw drone
        pygame.draw.rect(self.screen, (0, 0, 255), (*self.drone_pos - 10, 20, 20))  # Blue square

        # Add labels
        font = pygame.font.Font(None, 24)  # Default font, size 24
        balloon_label = font.render("Balloon", True, (0, 0, 0))  # Black text for balloon
        drone_label = font.render("Drone", True, (0, 0, 0))  # Black text for drone

        # Position labels near objects
        self.screen.blit(balloon_label, (self.balloon_pos[0] - 20, self.balloon_pos[1] - 20))
        self.screen.blit(drone_label, (self.drone_pos[0] - 20, self.drone_pos[1] - 20))
        
        # Display timer
        time_remaining = max(0, 30 - self.current_step / 30)  # 30 FPS assumed
        timer_label = font.render(f"Time Remaining: {time_remaining:.1f}s", True, (0, 0, 0))
        self.screen.blit(timer_label, (10, 10))
        
        # Display the score
        score_label = font.render(f"Score: {self.score}", True, (0, 0, 0))
        self.screen.blit(score_label, (10, 40))  # Display below the timer
        
        pygame.display.flip()
        self.clock.tick(30)

    def close(self):
        if self.render_mode == 'human':
            import pygame
            pygame.quit()

    def _get_obs(self):
        return np.concatenate((self.drone_pos, self.drone_vel, self.balloon_pos)).astype(np.float32)


### To train the PPO model

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Create vectorized environment for parallel training
env = make_vec_env(lambda: DroneChaseEnv(), n_envs=4)

# Define the PPO model
model = PPO("MlpPolicy", env, verbose=1)

# Train the model
model.learn(total_timesteps=800000)

# Save the model
model.save("ppo_drone_chase")


### To test and evaluate the model

In [3]:
import pygame
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Load the trained model
model = PPO.load("ppo_drone_chase")

# Create the environment with rendering enabled
env = DroneChaseEnv(render_mode='human')

# Reset the environment
observation, info = env.reset()

#num_steps = 300  # Simulate for 300 steps (10 seconds at 30 FPS)
num_steps = 30 * 30  # 30 seconds * 30 FPS = 900 steps

done = False
total_reward = 0.0

#while not done:
for _ in range(num_steps):

    # Get action from the model
    action, _ = model.predict(observation, deterministic=True)
    
    # Step the environment
    observation, reward, done, truncated, info = env.step(action)
    total_reward += reward

# Close the environment
env.close()

print(f"Total Reward: {total_reward}")

Total Reward: -1308.3312060093917


### To test and evaluate the model and show the plot result

In [3]:
import pygame
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import csv
import matplotlib.pyplot as plt
import numpy as np

# Load the trained model
model = PPO.load("ppo_drone_chase")

# Parameters for evaluation
#num_steps = 300  # Simulate for 300 steps (10 seconds at 30 FPS)
num_steps = 30 * 30  # 30 seconds * 30 FPS = 900 steps

rewards = []  # Store rewards for each step
scores = []  # Store scores (number of catches)
steps = list(range(num_steps))  # Step indices

# Create the environment with rendering enabled
env = DroneChaseEnv(render_mode='human')

# Reset the environment
observation, info = env.reset()

done = False
total_reward = 0.0

#while not done:
for _ in range(num_steps):

    # Get action from the model
    action, _ = model.predict(observation, deterministic=True)
    
    # Step the environment
    observation, reward, done, truncated, info = env.step(action)
    total_reward += reward
    
    # Log reward and score
    rewards.append(reward)
    scores.append(env.score)
    
# Save results to CSV
csv_file = "reward_performance.csv"
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Step", "Reward", "Score"])
    for step, reward, score in zip(steps, rewards, scores):
        writer.writerow([step, reward, score])

# Plot reward performance
plt.plot(steps, rewards, label="Reward")
plt.xlabel("Step")
plt.ylabel("Reward")
plt.title("Reward Performance Over Time")
plt.legend()
plt.grid()
plt.show()

# Plot cumulative score performance
plt.plot(steps, scores, label="Score", color="orange")
plt.xlabel("Step")
plt.ylabel("Score")
plt.title("Cumulative Score Over Time")
plt.legend()
plt.grid()
plt.show()

# Close the environment
env.close()

print(f"Total Reward: {total_reward}")

error: display Surface quit